In [2]:
import time
import Levenshtein as LV
import csv
import os
from tkinter import filedialog
from tkinter import *
from tkinter import filedialog,ttk
import tkinter as tk
#===============================================================
#========================= define functions ==============================
def BClocate(BC_seq,template_seq,valmin): #输入序列var_seq,查找在模板序列template_seq中的位置,返回实际位数; vakmin为允许的差异最大值
    out = -1
    for k in range(len(template_seq)-len(BC_seq)):
        val = LV.distance(BC_seq,template_seq[k:k+len(BC_seq)])
        valmin = min(val,valmin)
        if valmin == val:
            out = k
    return out
#========================= define functions ==============================

#========================= define GUI functions ==============================
def preview_fastq():
    file_path_list = filedialog.askopenfilenames(filetypes=[("txt files", "*.txt")])
    first_five_lines = ''
    count = 0
    for i in range(len(file_path_list)):
        with open(file_path_list[i], 'r',buffering = 1) as f1:
            first_five_lines = first_five_lines +'File '+str(i+1)+':\n'
            for line in f1:
                first_five_lines += str(count+1)+ '. '+line
                count += 1
                if count == 5:
                    count = 0
                    break
    file_path_entry.delete(0, tk.END)
    f_show = str(file_path_list).strip('(), \n').replace("'",'')
    file_path_entry.insert(0, f_show)
    text_box.delete('1.0', tk.END)
    text_box.insert(tk.END, first_five_lines)
    global file_list
    file_list = file_path_list
def show_default_BC():
    barcode1_entry.delete(0, tk.END)
    barcode1_entry.insert(0,'CTCCTTAC,TATGCAGT,TACTCCTT,AGGCTTAG,ATTAGACG,ATAGCCTT,TAAGGCTC,TCGCATAA')
def show_default_CS():
    cb_entry.delete(0, tk.END)
    cb_entry.insert(0,'TGGCGCTGAA')
#####################################################################################################
#####################################################################################################
def submit():
    global file_list
    file_paths = file_list
    barconsv = cb_entry.get().upper()
    barlist = barcode1_entry.get().upper().split(',')
    barlength = len(barlist[0])
    start0 = time.process_time()
    for file in file_paths:
        start = time.process_time()
        print('>> Processing file: '+file)
        mismatchdata = []
        datalist = []
        for idx in range(len(barlist)):
            datalist.append([])
        i = 1
        with open(file,'r',buffering = 1) as f:
            for row in f:
                seq = row.strip(' (),;\n')
                try:
                    seq = seq[:seq.index(' ')]
                except:
                    True
                try: # if able to locate barcode 2
                    barstart = seq.rindex(barconsv)+10
                    barseq = seq[barstart:barstart+barlength]
                    try: # if able to find barcode seq in barcode list and assign the sequence
                        idx = barlist.index(barseq)
                        datalist[idx].append([seq])
                    except: # if barcode seq is not in barcode list
                        mismatchdata.append([seq,'barcode not found'])
                except: # if not able to locate barcode 2
                    mismatchdata.append([seq,'conserved seq not found'])                
                if i%80000 == 0:
                    print('>> Sorted '+str(int(i/10000))+'0000 sequences.')
                    csvfile = open(file.replace('.txt','_mismatched data.txt'),'a',newline='')
                    write = csv.writer(csvfile)
                    write.writerows(mismatchdata)
                    mismatchdata = []
                    for k in range(len(barlist)):
                        csvfile = open(file.replace('.txt','_Barcode_')+barlist[k]+'.txt','a',newline='')
                        write = csv.writer(csvfile)
                        write.writerows(datalist[k])
                        datalist[k] = []
                i += 1
            csvfile = open(file.replace('.txt','_mismatched data.txt'),'a',newline='')
            write = csv.writer(csvfile)
            write.writerows(mismatchdata)
            for k in range(len(barlist)):
                csvfile = open(file.replace('.txt','_Barcode_')+barlist[k]+'.txt','a',newline='')
                write = csv.writer(csvfile)
                write.writerows(datalist[k])
        print('>> '+file+' Sorting finished.')
        print('>> Sorted '+str(i)+' sequences.')
        end = time.process_time()
        print('>> Processing time:' +str(end-start)+' s')
    f.close()
    csvfile.close()
    end0 = time.process_time()
    print('>> All files sorting finished.')
    print('>> Total processing time:' +str(end0-start0)+' s')
    result_label.config(text="All files sorting finished.")
    
#####################################################################################################
#####################################################################################################
def submit2():
    global file_list
    file_paths = file_list
    barconsv = cb_entry.get().upper()
    barlist = barcode1_entry.get().upper().split(',')
    barlength = len(barlist[0])
    start0 = time.process_time()
    for file in file_paths:
        start = time.process_time()
        mmfile = file.replace('.txt','_mismatched data.txt')
        mmfile_tmp = file.replace('.txt','_mismatched data_tmp.txt')
        os.renames(mmfile,mmfile_tmp)
        print('>> Processing file: '+ mmfile)
        mismatchdata = []
        datalist = []
        for idx in range(len(barlist)):
            datalist.append([])
        i = 1
        tcount = 0
        with open(mmfile_tmp,'r',buffering = 1) as f:
            for row in f:
                i += 1
                seq = row.strip(' (),;\n').split(',')[0]
                reason = row.strip(' (),;\n').split(',')[1]
                row_out = [seq,reason]
                if reason == 'conserved seq not found':
                    barstart = BClocate(barconsv,seq,2) # allow 2 bases in the ~10 bp conserve seq to be mutated
                    if barstart == -1:
                        mismatchdata.append(row_out)
                        continue
                    barseq = seq[barstart:barstart+barlength]
                    try:
                        idx = barlist.index(barseq)
                        datalist[idx].append([seq])
                        tcount += 1
                    except:
                        st = 1.5
                        for idx in range(len(barlist)):
                            distance =  LV.distance(barseq,barlist[idx])
                            if distance <= st:
                                datalist[idx].append([seq])
                                tcount += 1
                                st = distance
                                break
                        if st == 1.5:
                            mismatchdata.append([seq,'conserved seq mutated + barcode not found'])
                elif reason == 'barcode not found':
                    barstart = seq.rindex(barconsv)+10
                    barseq = seq[barstart:barstart+barlength]
                    st = 1.5
                    for idx in range(len(barlist)):
                        distance =  LV.distance(barseq,barlist[idx])
                        if distance <= st:
                            datalist[idx].append([seq])
                            tcount += 1
                            st = distance
                            break
                    if st == 1.5:
                        mismatchdata.append([seq,'barcode not found'])
                else: # other reasons from (files that has experienced ambiguous search)
                    seq = row.strip(' (),;\n').split(',')[0]
                    reason = row.strip(' (),;\n').split(',')[1]
                    row_out = [seq,reason]
                    mismatchdata.append([seq,row_out])
                if i%10000 == 0:
                    print('>> Sorted '+str(i)+' sequences.')
                    csvfile = open(mmfile,'a',newline='')
                    write = csv.writer(csvfile)
                    write.writerows(mismatchdata)
                    mismatchdata = []
                    for k in range(len(barlist)):
                        csvfile = open(file.replace('.txt','_Barcode_')+barlist[k]+'.txt','a',newline='')
                        write = csv.writer(csvfile)
                        write.writerows(datalist[k])
                        datalist[k] = []
            csvfile = open(mmfile,'a',newline='')
            write = csv.writer(csvfile)
            write.writerows(mismatchdata)
            for k in range(len(barlist)):
                csvfile = open(file.replace('.txt','_Barcode_')+barlist[k]+'.txt','a',newline='')
                write = csv.writer(csvfile)
                write.writerows(datalist[k])
        f.close()
        os.remove(mmfile_tmp)
        print('>> '+mmfile+' Sorting finished.')
        print('>> Sorted '+str(i)+' sequences.')
        print('>> Recognised and transferred '+str(tcount)+ ' sequences from mismatched data to barcode files.')
        end = time.process_time()
        print('>> Processing time:' +str(end-start)+' s')
    csvfile.close()
    end0 = time.process_time()
    print('>> All files sorting finished.')
    print('>> Total processing time:' +str(end0-start0)+' s')
    result2_label.config(text="All files ambiguous searching finished.")
    
#========================= define GUI functions ==============================

#====================== GUI interface ====================================
# Create the main window
root = tk.Tk()
root.title("NGS peptide analysis - S2 Second Barcode Sorting Tool")

# Create entry field and button for input data file selection dialogue
file_path_entry = tk.Entry(root, width = 90)
file_path_entry.grid(row=0, column=0, columnspan=6, padx=10, pady=5)
select_file_button = tk.Button(root, text="    Select Files and Preview   ", command=preview_fastq)
select_file_button.grid(row=0, column=6, columnspan = 2, padx=10, pady=5)

# Create a text box to display the first 5 lines of the text file
text_box = tk.Text(root, height=7, width=125)
text_box.grid(row=3, column=0, columnspan=8, padx=10, pady=5)

# Create labels and entry fields for Barcode 2
barcode1_label = tk.Label(root, text="Second Barcodes (BC2s, split with comma), e.g., CTCCTTAC,TATGCAGT,TACTCCTT ")
barcode1_label.grid(row=4, column=0, columnspan=6, padx=10, pady=5)
defaultbc = tk.Button(root,text = '  Use Default Barcode Set  ', command = show_default_BC)
defaultbc.grid(row=4, column=6, columnspan=2, padx=10, pady=5)
barcode1_entry = tk.Entry(root, width= 125)
barcode1_entry.grid(row=5, column=0, columnspan=8, padx=10, pady=5)


hyfen = tk.Label(root,text = '* ----------- * ----------- * ----------- * ----------- * ----------- * ----------- *\
----------- * ----------- * ----------- * ----------- * ----------- * ----------- * ----------- * ')
hyfen.grid(row=7,column = 0, columnspan = 8, padx = 10, pady = 5)

# Create a text box to display default seq format
tpl_box = tk.Text(root, height=4, width=100)
tpl_box.insert(tk.END,'   BC 1           Conserved 1                Library          Conserved 2     BC 2    Conserved 3 \n\
 ======== ---------------------------- ==================== --------------- ======== ------------\n\
 AGGAGTCC ATTCTATGCGGCCCAGCCGGCCATGGCG NNKTGC (NNK)n TGTNNK GGTTCTGGCGCTGAA ATAGCCTT CCTGTCTCTTAT\n           |F||Y||A||A||Q||P||A||M||A| |X||C|  |X|n  |C||X| |G||S||G||A||E|')
tpl_box.grid(row = 8, column = 2, columnspan = 6, rowspan = 3, padx=15)
tpl_label = tk.Label(root,text = 'The default format\n of a sequence:')
tpl_label.grid(row = 8, column =0, columnspan =2, rowspan = 3)

hyfen2 = tk.Label(root,text = '* ----------- * ----------- * ----------- * ----------- * ----------- * ----------- *\
----------- * ----------- * ----------- * ----------- * ----------- * ----------- * ----------- * ')
hyfen2.grid(row=11,column = 0, columnspan = 8, padx = 10, pady = 5)

# Create Entries for conserved sequences for selection of barcode
cb_label = tk.Label(root, text='      Conserved Seq 2 (~10 bp before BC 2)' )
cb_label.grid(row=12, column=0, columnspan=3, padx=5, pady=5)
cb_entry = tk.Entry(root,width = 30)
cb_entry.grid(row=12, column=3, columnspan=2, padx=5, pady=5)
ca_label2 = tk.Label(root, text='    XXXXXXXX (BC 2)' )
ca_label2.grid(row=12, column=4, columnspan=2, padx=5, pady=5)
defaultcs = tk.Button(root,text = '  Use Default Conserved Seq  ', command = show_default_CS)
defaultcs.grid(row=12, column=6, columnspan=2, padx=10, pady=5)

# Create a button to submit the input
submit_button = tk.Button(root, text=">>>>> Step 1: Submit and Sort <<<<<", width = 60, command=submit)
submit_button.grid(row=14, column=0, columnspan=4, padx=10, pady=10)
# Create a button to submit ambiguous search
submit2_button = tk.Button(root, text=">>>> Step 2: Ambiguous Search (Optional) <<<<", width = 60, command=submit2)
submit2_button.grid(row=15, column=0, columnspan=4, padx=10, pady=10)
# Create a label to display computing status
result_label = tk.Label(root, text="")
result_label.grid(row=14, column=4, columnspan=3, pady=10)
result2_label = tk.Label(root, text="")
result2_label.grid(row=15, column=4, columnspan=3, pady=20)
root.mainloop()
#====================== GUI interface ====================================

>> Processing file: C:/Users/86188/Desktop/error prone/Example/Example/Barcode_AGACTC.txt
>> C:/Users/86188/Desktop/error prone/Example/Example/Barcode_AGACTC.txt Sorting finished.
>> Sorted 2603 sequences.
>> Processing time:0.015625 s
>> Processing file: C:/Users/86188/Desktop/error prone/Example/Example/Barcode_ATCGCA.txt
>> C:/Users/86188/Desktop/error prone/Example/Example/Barcode_ATCGCA.txt Sorting finished.
>> Sorted 3298 sequences.
>> Processing time:0.0 s
>> All files sorting finished.
>> Total processing time:0.015625 s
>> Processing file: C:/Users/86188/Desktop/error prone/Example/Example/Barcode_AGACTC_mismatched data.txt
>> C:/Users/86188/Desktop/error prone/Example/Example/Barcode_AGACTC_mismatched data.txt Sorting finished.
>> Sorted 1158 sequences.
>> Recognised and transferred 28 sequences from mismatched data to barcode files.
>> Processing time:0.046875 s
>> Processing file: C:/Users/86188/Desktop/error prone/Example/Example/Barcode_ATCGCA_mismatched data.txt
>> C:/U

Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\86188\Anaconda3\lib\tkinter\__init__.py", line 1705, in __call__
    return self.func(*args)
  File "<ipython-input-2-a11af6892b50>", line 161, in submit2
    barstart = seq.rindex(barconsv)+10
ValueError: substring not found
